# Lab 7 - Checksum Recalculation and Packet Deparsing

<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. It describes how to recompute the checksum of a header. Recomputing the checksum is necessary if the packet header was modified by the P4 program. The lab also describes how a P4 program performs deparsing to emit headers.
<figure style="text-align: center;">
<img src="./labs_files/lab6/figs/0_00_fabric_topology.png" width="700px"><br>

# Introduction

## Deparsing

The P4 program includes a deparser that specifies which headers are to be emitted. The deparser emits the headers and the payload of the original packet. Note that only the valid headers are emitted. A header is considered valid after it has been parsed in the P4 program or after the program explicitly validates the header. 

## Checksums
<div style="text-align: justify;">
Several protocols use checksums to validate the integrity of the packet headers. A checksum is a small value derived from another data block, often through a checksum algorithm such as the Cyclic Redundancy Check (CRC). The checksum calculation and verification start with the sender calculating the checksum of the data before transmitting the packet. Then, the checksum value is inserted into the packet header. Upon receiving the packet, the receiver computes the checksum of the received packet using the same algorithm as the one used by the sender. If the calculated checksum value matches the one in the packet header, the packet is verified; otherwise, a transmission error has occurred. Incorrect checksums typically lead to dropping the packet by the switch. </div><br>
<div style="text-align: justify;">
In a P4 program, the developer may change the packet headers. For example, if the program is implementing a routing function, then header fields such as the Time-to-Live (TTL) must be modified. Any change to the header fields will cause the checksum value to change. Therefore, it is necessary to recompute the checksum in the P4 program in case modifications are made to the header fields. </div><br>
<div style="text-align: justify;">
Figure 1 shows an example of computing the checksum and implementing a deparser in a P4 program. The deparser is defined as a control block and is executed after finishing the packet processing by the other control blocks.</div><br>

<figure style="text-align: center;">
  <img src="./labs_files/lab7/figs/intro_01.png" width="800" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Runtime management of a P4 target (DPDK).</figcaption>
</figure>

In the PNA [<a href="#References">1</a>] architecture, the checksum update is implemented in the deparser control block before the packet headers are emitted. In like 7, the ```InternetChecksum()``` instance is needed to create a checksum object with the name checksum. The checksum update is then done in the apply block of the deparser. 

Note that in DPDK the checksum is calculated using the 16-bit one’s complement algorithm [<a href="#References">2,3</a>]. Therefore, the data whose checksum is to be computed should be introduced as words of 16 bits or 32 bits. The data usually includes the header fields of the protocol which uses the checksum. The example above shows the header fields of IPv4. In lines 11 and 12 the ```++``` operator is used to concatenate the packet headers into a 16-bit word and a 32-bit word. In line 13, the ```checksum.add()``` function takes the data which includes the header fields in this example. Finally, in line 14, the checksum field in the packet is updated using the ```checksum.get()``` function.

The deparser control block has a ```packet_out``` type in its parameters. The ```packet_out``` type includes the ```emit``` method which accepts the headers to be reassembled when the deparser constructs the outgoing packet (lines 17-19). Note that the order of emitting packets' headers is important, and the headers are only emitted if they are valid.

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab7_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab7_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is NCSA


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 14, Time: 306 sec


ID,7e63ec74-600c-4046-a0f8-0ae4bde17522
Name,P4DPDK_lab7_vnic_test
Lease Expiration (UTC),2024-09-23 04:08:25 +0000
Lease Start (UTC),2024-09-22 04:08:25 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8ae59271-b3e3-4ad6-9b20-b6b10ef61dce,server1,4,8,100,default_ubuntu_20,qcow2,ncsa-w1.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fed4:a51c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fed4:a51c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [8]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools syslinux-utils', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja scapy', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [56]:
stdout, stderr = server1.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server1.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server1.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

# Step 5: Writing and compiling the P4 code
This section demonstrates how to implement the packet deparser in P4.

## Step 5.1: Programming the deparser block

Click on [deparser.p4](./labs_files/lab7/deparser.p4) to open the code in the editor.

<img src="./labs_files/lab7/figs/5_01_01.png" width="750px"><br>

We can see that the deparser.p4 declares a control block named MyDeparser. Note that the body of the apply block of the deparser is empty for us to fill it. 

<hr>

We will start by emitting the ethernet header by typing the following line of code.

    packet.emit(hdr.ethernet)

<img src="./labs_files/lab7/figs/5_01_02.png" width="700"><br>

<hr>

Emit the IPv4 header by typing the following line of code.

    packet.emit(hdr.ipv4)

<img src="./labs_files/lab7/figs/5_01_03.png" width="700"><br>

<hr>

Emit the TCP header by typing the following line of code.

    packet.emit(hdr.tcp)

<img src="./labs_files/lab7/figs/5_01_04.png" width="700"><br>

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.2: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [57]:
server1.upload_file('labs_files/lab7/headers.p4','headers.p4')
server1.upload_file('labs_files/lab7/parser.p4','parser.p4')
server1.upload_file('labs_files/lab7/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab7/control.p4','control.p4')
server1.upload_file('labs_files/lab7/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab7/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab7.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 control.p4
deparser.p4
dpdk
ethdev.io
headers.p4
lab7.cli
lab7.spec
main.p4
nat64.sh
p4c
parser.p4
precontrol.p4
rules_exact.txt
rules_lpm.txt
run_pipeline.sh
sender.py
set_topology.sh


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab7.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

We will also upload the rules.txt file to load and add rules to the forwarding table. The rules.txt file contains rules for exact matching.

In [ ]:
server1.upload_file('labs_files/lab7/lab7.cli','lab7.cli')
server1.upload_file('labs_files/lab7/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab7/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab7/set_topology.sh','set_topology.sh')
server1.upload_file('labs_files/lab7/sender.py','sender.py')
server1.upload_file('labs_files/lab7/rules_exact.txt','rules_exact.txt')
server1.upload_file('labs_files/lab7/rules_lpm.txt','rules_lpm.txt')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x sender.py')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab7/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [58]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fed4:a51c'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab7/figs/6_03.png" width="750px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [42]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab7/figs/6_04_ip.png" width="650px"><br>

# Step 7: Sending and capturing a packet

### Step 7.1: Opening a new terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab7/figs/7_01_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste into the terminal to enter to server1.

In [47]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fef0:62'

### Step 7.2: Starting packet capturing at the interface of h3
Issue the command tcpdum to start a sniffer in host 3.

    sudo ip netns exec h3 tcpdump -i dtap2 -vvv

<img src="./labs_files/lab7/figs/7_02.png" width="700px"><br>

The ```-i``` parameter in the tcpdump command, represents the interface on which the packets are being captured. In this application, we will be sending a packet from host 1 to host 3 and capture the packet received at the interface on host 3 (dtap2). The ```-vvv``` parameter increases verbosity to display the captured packet details.


### Step 7.3: Sending packets from h1 to h3
Now, we will send a packet from h1 to h3 over the destination IP address 192.168.30.1 by running the provided Python script sender.py.

In [43]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.30.1')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.30.1

Sent 1 packets.


Running the Python script requires two parameters:

•	```-s```: Sender (h1 or h2 or h3) <br>
•	```-d```: Destination IP Address 

### Step 7.4: Inspect the pipeline terminal
Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab7/figs/7_04.png" width="700px"><br>

The “emit header” log appears three times in the grey box, corresponds to the deparsed ethernet header (header 0) with size of 14 bytes (112 bits), the deparsed IPv4 header (header 1) with size 20 bytes (160 bits) and the deparsed TCP header (header 2) with size 20 bytes (160 bits) as declared in the deparser.p4 file. Note that the order of the deparsed headers matches the order of the parsed headers as shown in the “extract header” logs. The “tx 1 pkt to port 2” log indicated that one packet has been sent to port 2 which corresponds to host 3.

### Step 7.5: Inspect the captured packet
Switch back to the terminal tab in which the tcpdump sniffer is running and press ```Ctrl+C```.

<img src="./labs_files/lab7/figs/7_05.png" width="900px"><br>

We can see that the checksum in that packet is incorrect resulting in a bad checksum warning highlighted in the grey box. This is because the P4 program changed the header field value (i.e., TTL), but did not update the checksum in the packet. 

### Step 7.6: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab7/figs/7_06.png" width="700px"><br>

Execute the command below to avoid having a pipeline instance running in the background.

In [59]:
stdout, stderr = server1.execute(f'sudo pkill -9 pipeline')

 sudo: unable to resolve host server1: Name or service not known
 

# Step 8: Implementing checksum update in P4

## Step 8.1: Programming the checksum update in the P4 code

Click on [deparser.p4](./labs_files/lab7/deparser.p4) to open the code in the editor.

<hr>

Start by creating the InternetChecksum object by typing the following line on code before the apply block in the deparser.p4 code.

    InternetChecksum() checksum;

<img src="./labs_files/lab7/figs/8_01_01.png" width="700"><br>

In this line, the ```InternetChecksum()``` function creates a checksum object given the name checksum.

<hr>

Concatenate the IPv4 header field in a 16-bit word and a 32-bit word by inserting the following code at the beginning of the apply block.
    
    bit<16> word1 = hdr.ipv4.version ++ hdr.ipv4.ihl ++ hdr.ipv4.diffserv;
    bit<32> word4 = hdr.ipv4.flags ++ hdr.ipv4.fragOffset ++ hdr.ipv4.ttl ++ hdr.ipv4.protocol;

<img src="./labs_files/lab7/figs/8_01_02.png" width="700"><br>

The two declared words word1 and word4 will be used in the checksum calculation process. The ```++``` operator in P4 is used to concatenate the header field and join them in one variable.

<hr>

Add to the checksum object the data considered for the checksum calculation by inserting the code below.

    checksum.add({
        word1,
        hdr.ipv4.totalLen,
        hdr.ipv4.identification,
        word4,
        hdr.ipv4.srcAddr,
        hdr.ipv4.dstAddr
    });

<img src="./labs_files/lab7/figs/8_01_03.png" width="700"><br>

<hr>

Update the checksum by inserting the following code.

    hdr.ipv4.hdrChecksum = checksum.get();

<img src="./labs_files/lab7/figs/8_01_04.png" width="700"><br>

The ```checksum.get()``` function returns the value of the checksum. In this line of code, the value of the updated checksum is written in the checksum field of the IPv4 header of the packet ```hdr.ipv4.hdrChechsum```.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 8.2: Compiling the P4 code
To upload the modified P4 module and compile main.p4, issue the following command.

In [30]:
server1.upload_file('labs_files/lab7/deparser.p4','deparser.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab7.spec')

 sudo: unable to resolve host server1: Name or service not known
 

The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab7.spec``` file which is a specification file needed to run the pipeline.

# Step 9: Re-running the P4-DPDK pipeline and the lab topology

## Step 9.1: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab7/figs/6_03.png" width="750px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 9.2: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [60]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab7/figs/6_04_ip.png" width="650px"><br>

# Step 10: Sending and capturing packets with checksum update

This concludes Lab 7. Please delete your slice when you are done with your experiment.

### Step 10.1: Starting packet capturing at the interface of h3
Issue the command tcpdum to start a sniffer in host 3.

    sudo ip netns exec h3 tcpdump -i dtap2 -vvv

<img src="./labs_files/lab7/figs/7_02.png" width="700px"><br>

The ```-i``` parameter in the tcpdump command, represents the interface on which the packets are being captured. In this application, we will be sending a packet from host 1 to host 3 and capture the packet received at the interface on host 3 (dtap2). The ```-vvv``` parameter increases verbosity to display the captured packet details.


### Step 10.2: Sending packets from h1 to h3
Now, we will send a packet from h1 to h3 over the destination IP address 192.168.30.1 by running the provided Python script sender.py.

In [61]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.30.1')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.30.1

Sent 1 packets.


Running the Python script requires two parameters:

•	```-s```: Sender (h1 or h2 or h3) <br>
•	```-d```: Destination IP Address 

### Step 10.3: Inspect the pipeline terminal
Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab7/figs/10_03.png" width="900px"><br>

The process of concatenating the header fields into words generates a lot of logs. We are mainly concerned with the logs that correspond to adding the data for the checksum calculation in the first grey box and updating the value in the checksum header field in the second grey box.

### Step 10.4: Inspect the captured packet
Switch back to the terminal tab in which the tcpdump sniffer is running and press ```Ctrl+C```.

<img src="./labs_files/lab7/figs/10_04.png" width="900px"><br>

We can see that we do not have a bad checksum warning indicating that the calculated checksum value is correct. This is because the P4 program changed the header field value (i.e., TTL), but did not update the checksum in the packet. 

### Step 10.5: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab7/figs/10_05.png" width="700px"><br>

Execute the command below to avoid having a pipeline instance running in the background.

In [21]:
stdout, stderr = server1.execute(f'sudo pkill -9 pipeline')

 sudo: unable to resolve host server1: Name or service not known
 

# Step 11: Delete the slice

This concludes Lab 7. Please delete your slice when you are done with your experiment.

In [62]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab7_vnic")
slice.delete()

# References

1.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html
2.	P4lang, “p4c core.p4”. [Online]. Available: https://github.com/p4lang/p4c/blob/main/p4include/core.p4.
3.	P4lang, “pna.p4”, [Online]. Available: https://github.com/p4lang/pna/blob/main/pna.p4